In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import random
import time
from sklearn.metrics import adjusted_rand_score, homogeneity_score, completeness_score
import tracemalloc

# Carregando a base de dados Iris
iris = load_iris()
# Criando um DataFrame do Pandas, desconsiderando a classe alvo
X = pd.DataFrame(iris.data, columns=iris.feature_names)

print("Visualização das 5 primeiras linhas da base de dados:")
print(X.head())

print("\nFormato da base de dados (instâncias, atributos):", X.shape)

In [ ]:
np.random.seed(42)

def calcular_distancia_euclidiana(ponto1, ponto2):
    """Calcula a distância Euclidiana entre dois pontos."""
    return np.sqrt(np.sum((ponto1 - ponto2)**2))

def inicializar_centroides(dados, k):
    """Inicializa os centróides escolhendo k pontos aleatórios dos dados."""
    n_amostras = dados.shape[0]
    indices_aleatorios = np.random.choice(n_amostras, k, replace=False)
    centroides = [dados[i] for i in indices_aleatorios]
    return centroides

def atribuir_clusters(dados, centroides):
    """Atribui cada ponto de dado ao centróide mais próximo."""
    clusters = [[] for _ in range(len(centroides))]
    for i, ponto in enumerate(dados):
        distancias = [calcular_distancia_euclidiana(ponto, centroide) for centroide in centroides]
        indice_centroide = np.argmin(distancias)
        clusters[indice_centroide].append(i)
    return clusters

def atualizar_centroides(dados, clusters):
    """Atualiza a posição dos centróides com base na média dos pontos do cluster."""
    novos_centroides = []
    for cluster in clusters:
        novo_centroide = np.mean([dados[i] for i in cluster], axis=0)
        novos_centroides.append(novo_centroide)
    return novos_centroides

def kmeans_hardcore(dados, k, max_iteracoes=100):
    """Executa o algoritmo K-means completo."""
    dados_np = dados.to_numpy()
    centroides = inicializar_centroides(dados_np, k)t

    for _ in range(max_iteracoes):
        clusters = atribuir_clusters(dados_np, centroides)
        centroides_antigos = centroides
        centroides = atualizar_centroides(dados_np, clusters)

        # Critério de parada: se os centróides não mudarem
        if np.allclose(np.array(centroides_antigos), np.array(centroides)):
            break

    # Criar um array de rótulos para cada ponto de dado
    labels = np.zeros(dados_np.shape[0])
    for i, cluster in enumerate(clusters):
        for indice_ponto in cluster:
            labels[indice_ponto] = i

    return labels, np.array(centroides)

print("Funções do K-means 'Hardcore' definidas com sucesso.")

In [ ]:
# Executando o K-means hardcore para k=3
k3_labels_hardcore, k3_centroides_hardcore = kmeans_hardcore(X, k=3)

k3_silhouette_hardcore = silhouette_score(X, k3_labels_hardcore)

print(f"Resultado da implementação Hardcore para K=3:")
print(f"Silhouette Score: {k3_silhouette_hardcore:.4f}")

# Executando o K-means hardcore para k=5
k5_labels_hardcore, k5_centroides_hardcore = kmeans_hardcore(X, k=5)

k5_silhouette_hardcore = silhouette_score(X, k5_labels_hardcore)

print(f"Resultado da implementação Hardcore para K=5:")
print(f"Silhouette Score: {k5_silhouette_hardcore:.4f}")

In [ ]:
# Inicializando e executando o KMeans do Sklearn para k=3
kmeans_sklearn_k3 = KMeans(n_clusters=3, random_state=42, n_init=10)
kmeans_sklearn_k3.fit(X)

k3_silhouette_sklearn = silhouette_score(X, kmeans_sklearn_k3.labels_)

print(f"Resultado da implementação com Scikit-learn para K=3:")
print(f"Silhouette Score: {k3_silhouette_sklearn:.4f}")

# Inicializando e executando o KMeans do Sklearn para k=5
kmeans_sklearn_k5 = KMeans(n_clusters=5, random_state=42, n_init=10)
kmeans_sklearn_k5.fit(X)

k5_silhouette_sklearn = silhouette_score(X, kmeans_sklearn_k5.labels_)

print(f"Resultado da implementação com Scikit-learn para K=5:")
print(f"Silhouette Score: {k5_silhouette_sklearn:.4f}")

In [ ]:
# Criando um DataFrame para comparar os resultados
comparacao_df = pd.DataFrame({
    'Implementação': ['Hardcore', 'Scikit-learn'],
    'Silhouette Score (K=3)': [k3_silhouette_hardcore, k3_silhouette_sklearn],
    'Silhouette Score (K=5)': [k5_silhouette_hardcore, k5_silhouette_sklearn]
})

print("Tabela de Comparação dos Scores:")
print(comparacao_df.to_string(index=False))

In [ ]:
# Definindo o melhor k encontrado
melhor_k = 3

# Reduzindo a dimensionalidade para 2 componentes
pca_2d = PCA(n_components=2)
X_pca_2d = pca_2d.fit_transform(X)

# Clusterizando os dados reduzidos com nossa implementação
labels_pca_2d, centroides_pca_2d_orig = kmeans_hardcore(pd.DataFrame(X_pca_2d), k=melhor_k)
# O centróide retornado está no espaço PCA, pronto para plotar

# Plotando os resultados
plt.figure(figsize=(10, 7))
sns.scatterplot(x=X_pca_2d[:, 0], y=X_pca_2d[:, 1], hue=labels_pca_2d, palette='viridis', s=100, alpha=0.7)
plt.scatter(centroides_pca_2d_orig[:, 0], centroides_pca_2d_orig[:, 1], s=300, c='red', marker='X', label='Centróides')
plt.title('Clusterização K-means após PCA (2 Componentes)')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Reduzindo a dimensionalidade para 1 componente
pca_1d = PCA(n_components=1)
X_pca_1d = pca_1d.fit_transform(X)

# Clusterizando os dados reduzidos com nossa implementação
labels_pca_1d, centroides_pca_1d_orig = kmeans_hardcore(pd.DataFrame(X_pca_1d), k=melhor_k)
# O centróide retornado está no espaço PCA, pronto para plotar

# Para a visualização 1D, criamos um eixo y com zeros
y_zeros = np.zeros(len(X_pca_1d))

# Plotando os resultados
plt.figure(figsize=(10, 4))
sns.scatterplot(x=X_pca_1d[:, 0], y=y_zeros, hue=labels_pca_1d, palette='viridis', s=100, alpha=0.7)
plt.scatter(centroides_pca_1d_orig[:, 0], np.zeros(len(centroides_pca_1d_orig)), s=300, c='red', marker='X', label='Centróides')
plt.title('Clusterização K-means após PCA (1 Componente)')
plt.xlabel('Componente Principal 1')
plt.yticks([]) # Remove o eixo y
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# --- Medindo o tempo da implementação Hardcore ---
start_time_hardcore = time.time()
kmeans_hardcore(X, k=3)
end_time_hardcore = time.time()
tempo_hardcore = end_time_hardcore - start_time_hardcore

# --- Medindo o tempo da implementação Scikit-learn ---
start_time_sklearn = time.time()
KMeans(n_clusters=3, random_state=42, n_init=10).fit(X)
end_time_sklearn = time.time()
tempo_sklearn = end_time_sklearn - start_time_sklearn

print("--- Comparação de Tempo de Execução (k=3) ---")
print(f"Implementação Hardcore: {tempo_hardcore:.6f} segundos")
print(f"Implementação Scikit-learn: {tempo_sklearn:.6f} segundos")

In [ ]:
# Carregando as classes verdadeiras (target)
y_true = iris.target

# Labels já calculados anteriormente para k=3
labels_hardcore = k3_labels_hardcore
labels_sklearn = kmeans_sklearn_k3.labels_

# Calculando as métricas para a implementação Hardcore
ari_hardcore = adjusted_rand_score(y_true, labels_hardcore)
homogeneity_hardcore = homogeneity_score(y_true, labels_hardcore)
completeness_hardcore = completeness_score(y_true, labels_hardcore)

# Calculando as métricas para a implementação Scikit-learn
ari_sklearn = adjusted_rand_score(y_true, labels_sklearn)
homogeneity_sklearn = homogeneity_score(y_true, labels_sklearn)
completeness_sklearn = completeness_score(y_true, labels_sklearn)

print("--- Comparação com as Classes Originais (k=3) ---")
print("\nImplementação Hardcore:")
print(f"Adjusted Rand Index (ARI): {ari_hardcore:.4f}")
print(f"Homogeneidade: {homogeneity_hardcore:.4f}")
print(f"Completude: {completeness_hardcore:.4f}")

print("\nImplementação Scikit-learn:")
print(f"Adjusted Rand Index (ARI): {ari_sklearn:.4f}")
print(f"Homogeneidade: {homogeneity_sklearn:.4f}")
print(f"Completude: {completeness_sklearn:.4f}")

In [ ]:
def formatar_memoria(size_bytes):
    """Formata bytes em uma string legível (KB ou MB)."""
    if size_bytes > 1024 * 1024:
        return f"{size_bytes / (1024 * 1024):.2f} MB"
    return f"{size_bytes / 1024:.2f} KB"

# --- Medindo o uso de memória da implementação Hardcore ---
tracemalloc.start()
kmeans_hardcore(X, k=3)
current_hardcore, peak_hardcore = tracemalloc.get_traced_memory()
tracemalloc.stop()

# --- Medindo o uso de memória da implementação Scikit-learn ---
tracemalloc.start()
KMeans(n_clusters=3, random_state=42, n_init=10).fit(X)
current_sklearn, peak_sklearn = tracemalloc.get_traced_memory()
tracemalloc.stop()

print("--- Comparação de Pico de Uso de Memória (k=3) ---")
print(f"Implementação Hardcore: {formatar_memoria(peak_hardcore)}")
print(f"Implementação Scikit-learn: {formatar_memoria(peak_sklearn)}")

In [ ]:
# Dados de tempo de execução (já calculados anteriormente)
implementacoes = ['Hardcore', 'Scikit-learn']
tempos = [tempo_hardcore, tempo_sklearn]

plt.figure(figsize=(8, 6))
bars = plt.bar(implementacoes, tempos, color=['skyblue', 'lightgreen'])

for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2.0, yval, f'{yval:.6f} s', va='bottom', ha='center')

plt.ylabel('Tempo de Execução (segundos)')
plt.title('Comparação de Tempo de Execução (k=3)')

if max(tempos) / min(tempos) > 10:
    plt.yscale('log')
    plt.ylabel('Tempo de Execução (segundos) - Escala Log')

plt.show()

# Dados das métricas (já calculados anteriormente)
metricas_nomes = ['Adjusted Rand Index', 'Homogeneidade', 'Completude']
scores_hardcore = [ari_hardcore, homogeneity_hardcore, completeness_hardcore]
scores_sklearn = [ari_sklearn, homogeneity_sklearn, completeness_sklearn]

x = np.arange(len(metricas_nomes))
width = 0.35

fig, ax = plt.subplots(figsize=(12, 7))
rects1 = ax.bar(x - width/2, scores_hardcore, width, label='Hardcore', color='skyblue')
rects2 = ax.bar(x + width/2, scores_sklearn, width, label='Scikit-learn', color='lightgreen')

ax.set_ylabel('Scores')
ax.set_title('Comparação de Métricas de Clusterização (k=3)')
ax.set_xticks(x)
ax.set_xticklabels(metricas_nomes)
ax.legend()
ax.set_ylim(0, 1.1)

ax.bar_label(rects1, padding=3, fmt='%.4f')
ax.bar_label(rects2, padding=3, fmt='%.4f')

fig.tight_layout()
plt.show()

# Dados de uso de memória em KB
memoria_hardcore_kb = peak_hardcore / 1024
memoria_sklearn_kb = peak_sklearn / 1024
memorias = [memoria_hardcore_kb, memoria_sklearn_kb]

plt.figure(figsize=(8, 6))
bars = plt.bar(implementacoes, memorias, color=['skyblue', 'lightgreen'])

for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2.0, yval, f'{yval:.2f} KB', va='bottom', ha='center')

plt.ylabel('Pico de Uso de Memória (KB)')
plt.title('Comparação de Uso de Memória (k=3)')
plt.show()